PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, Upstage)

In [ ]:
# %pip install -q openai
# %pip install -q langchain
# %pip install -q -U langchain-openai

In [1]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:4])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

gsk_
WA


In [ ]:
# %pip install -q pypdf
# %pip install -q faiss-cpu
# %pip install -q tiktoken

In [14]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [7]:
# PDF 로드
loader = PyPDFLoader("../data/tutorial-korean.pdf")
documents = loader.load()
print(len(documents))

39


In [17]:
# 텍스트 분할
#text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, 
    chunk_overlap=0,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)
#texts = text_splitter.split_documents(documents)
texts = loader.load_and_split(text_splitter=text_splitter)
print(len(texts))

284


In [18]:
from langchain_upstage import UpstageEmbeddings

# OpenAI Embeddings 적용
#embeddings = OpenAIEmbeddings()
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# FAISS 벡터 저장소 생성
vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local("../db/faiss_db")
# 벡터 저장소에서 검색하기
# search_type="similarity" 옵션은 retrieval 객체에서 유사성 검색을 사용하여 질문 vector와 가장 유사한 문장 vector를 선택함
# search_kwargs 옵션은 vector 저장소에서 Prompt 2개의 텍스트 덩어리로 보내려는 것을 의미함
retriever = vector_store.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}
)

In [20]:
from langchain_openai import ChatOpenAI
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA

# ChatOpenAI는 기본모델인  gpt-3.5-turbo 사용하고
# temperature=0은 보수적인 지문에 대한 답변을 내고, temperature=1 다양한 답변을 낼 수 있음 
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
)

In [21]:
from langchain.prompts import PromptTemplate

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")
print(prompt)

 프롬프트 설정 완료
input_variables=['context', 'question'] input_types={} partial_variables={} template='\n당신은 BlueJ 프로그래밍 환경 전문가입니다. \n아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.\n\n문서 내용:\n{context}\n\n질문: {question}\n\n답변 규칙:\n1. 문서 내용만을 근거로 답변하세요\n2. 단계별 설명이 필요하면 순서대로 작성하세요  \n3. 구체적인 메뉴명, 버튼명을 포함하세요\n4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요\n\n답변:'


In [22]:

# RetrievalQA가 실질적인 RAG를 수행하는 객체
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # 기존 retriever 유지
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n당신은 BlueJ 프로그래밍 환경 전문가입니다. \n아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.\n\n문서 내용:\n{context}\n\n질문: {question}\n\n답변 규칙:\n1. 문서 내용만을 근거로 답변하세요\n2. 단계별 설명이 필요하면 순서대로 작성하세요  \n3. 구체적인 메뉴명, 버튼명을 포함하세요\n4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요\n\n답변:'), llm=ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x000001A2708EA750>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001A2708EADB0>, model_name='solar-pro', temperature=0.5, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1'), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template=

In [23]:
query = "코드패드는 무엇이고 어떻게 사용하나요?"
query_result = chain.invoke(query)
print(type(query_result))
print(query_result)

<class 'dict'>
{'query': '코드패드는 무엇이고 어떻게 사용하나요?', 'result': '**코드패드란?**  \nBlueJ 코드패드는 자바 코드(표현식 또는 명령문)의 일부분을 **빠르고 간편하게 실행/평가**할 수 있는 도구입니다. 프로그램 코드의 동작을 테스트하거나 구문을 예증하는 데 유용합니다.  \n\n---\n\n### **코드패드 사용 방법 (단계별)**  \n1. **코드패드 열기**  \n   - BlueJ 메인 화면에서 **보기(Show) 메뉴** → **"코드패드 보기(Show Code Pad)"**를 선택합니다.  \n   - 또는 단축키 `Ctrl+Alt+C`(Windows/Linux) 또는 `Cmd+Option+C`(Mac)를 사용할 수 있습니다. *(단, 문서에 명시되지 않은 단축키는 제외)*  \n\n2. **코드 입력 및 실행**  \n   - 코드패드 영역에 자바 표현식 또는 명령문을 입력합니다.  \n     - 예: `2 + 3`, `System.out.println("Hello")`  \n   - **Enter 키**를 눌러 코드를 실행합니다. 결과는 화면 아래에 표시됩니다.  \n\n3. **이전 입력 재사용**  \n   - **상하 화살표 키**를 눌러 이전에 입력한 코드 이력을 확인하고 재사용할 수 있습니다.  \n   - 이력 내용을 수정한 후 다시 실행할 수도 있습니다.  \n\n4. **기능 제한 사항**  \n   - 코드패드는 **클래스 정의나 전체 프로그램은 실행할 수 없으며**, 단일 표현식/명령문만 평가 가능합니다.  \n   - 문서에서 명시되지 않은 추가 기능(예: 디버깅)은 지원하지 않습니다.  \n\n---  \n**참고**: 문서에 없는 정보(예: 단축키, 클래스 실행 가능 여부)는 "문서에서 찾을 수 없습니다"로 답변합니다.', 'source_documents': [Document(id='85c6f820-041e-4431-985a-4f37bb069775', metadata

In [24]:
query_result['result']

'**코드패드란?**  \nBlueJ 코드패드는 자바 코드(표현식 또는 명령문)의 일부분을 **빠르고 간편하게 실행/평가**할 수 있는 도구입니다. 프로그램 코드의 동작을 테스트하거나 구문을 예증하는 데 유용합니다.  \n\n---\n\n### **코드패드 사용 방법 (단계별)**  \n1. **코드패드 열기**  \n   - BlueJ 메인 화면에서 **보기(Show) 메뉴** → **"코드패드 보기(Show Code Pad)"**를 선택합니다.  \n   - 또는 단축키 `Ctrl+Alt+C`(Windows/Linux) 또는 `Cmd+Option+C`(Mac)를 사용할 수 있습니다. *(단, 문서에 명시되지 않은 단축키는 제외)*  \n\n2. **코드 입력 및 실행**  \n   - 코드패드 영역에 자바 표현식 또는 명령문을 입력합니다.  \n     - 예: `2 + 3`, `System.out.println("Hello")`  \n   - **Enter 키**를 눌러 코드를 실행합니다. 결과는 화면 아래에 표시됩니다.  \n\n3. **이전 입력 재사용**  \n   - **상하 화살표 키**를 눌러 이전에 입력한 코드 이력을 확인하고 재사용할 수 있습니다.  \n   - 이력 내용을 수정한 후 다시 실행할 수도 있습니다.  \n\n4. **기능 제한 사항**  \n   - 코드패드는 **클래스 정의나 전체 프로그램은 실행할 수 없으며**, 단일 표현식/명령문만 평가 가능합니다.  \n   - 문서에서 명시되지 않은 추가 기능(예: 디버깅)은 지원하지 않습니다.  \n\n---  \n**참고**: 문서에 없는 정보(예: 단축키, 클래스 실행 가능 여부)는 "문서에서 찾을 수 없습니다"로 답변합니다.'

In [26]:
query_result['source_documents']

[Document(id='85c6f820-041e-4431-985a-4f37bb069775', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, page_content='6. 코드패드 사용하기 ··················································· ······························································· ········· 21'),
 Document(id='e0613533-3e9b-4ee2-9289-cb3d2021e8ba', metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 20, 'page_label': '21'}, page_

In [25]:
for key in query_result.keys():
    print(key)

query
result
source_documents


In [27]:
print(type(query_result['source_documents']))
type(query_result['source_documents'][0])

<class 'list'>


langchain_core.documents.base.Document

In [28]:
doc = query_result['source_documents'][0]
print(type(doc))
doc_dict = doc.model_dump()
print(type(doc_dict))
print(doc_dict)

<class 'langchain_core.documents.base.Document'>
<class 'dict'>
{'id': '85c6f820-041e-4431-985a-4f37bb069775', 'metadata': {'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, 'page_content': '6. 코드패드 사용하기 ··················································· ······························································· ········· 21', 'type': 'Document'}


In [29]:
for k in doc_dict:
    print(k)

id
metadata
page_content
type


In [30]:
doc_dict['metadata']['title']

'<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>'

In [31]:
query = "애플릿을 만들고 실행하는 방법을 설명해주세요"
query_result = chain.invoke(query)
print(query_result)

{'query': '애플릿을 만들고 실행하는 방법을 설명해주세요', 'result': '### BlueJ에서 애플릿 만들기 및 실행 방법 (문서 내용 기반)\n\n#### 1. 애플릿 만들기\n1. **새 클래스 생성**  \n   - BlueJ 메뉴에서 **`File > New Class`**를 선택하거나 툴바의 **`New Class` 버튼**을 클릭합니다.  \n   - **`New Class` 대화상자**에서:  \n     - 클래스 이름을 입력합니다.  \n     - **`Type`** 드롭다운 메뉴에서 **`Applet`**을 선택합니다.  \n     - **`OK`**를 클릭하여 애플릿 기본 골격을 생성합니다.  \n   - 생성된 클래스는 `java.applet.Applet`을 상속받는 기본 코드로 구성됩니다.\n\n2. **애플릿 코드 작성**  \n   - 생성된 클래스에 애플릿 동작을 정의하는 메서드(예: `paint(Graphics g)`)를 추가합니다.  \n   - 문서에 구체적인 코드 예시는 없으나, 일반 클래스와 동일한 방식으로 편집 가능합니다.\n\n---\n\n#### 2. 애플릿 실행 방법\n1. **예제 프로젝트 활용 (권장)**  \n   - **`examples` 디렉토리**에서 **`appletdemo` 프로젝트**를 엽니다.  \n   - 이 프로젝트는 미리 구성된 애플릿 예제를 포함합니다.  \n\n2. **직접 만든 애플릿 실행**  \n   - 애플릿 클래스를 컴파일합니다(**`Compile` 버튼** 또는 우클릭 후 **`Compile`**).  \n   - 컴파일 후, 애플릿 클래스를 더블클릭하거나 우클릭 메뉴에서 **`Run`**을 선택합니다.  \n   - BlueJ는 내장된 HTML 뷰어를 통해 애플릿을 실행합니다.  \n     - 실행 시 애플릿이 표시될 HTML 파일이 자동으로 생성되며, 별도의 웹 브라우저 없이 BlueJ 내에서 확인 가능합니다.\n\n---\n\n#### 3. 추가 참고 사

In [ ]:
print(query_result['result'])

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

system_template = """Use the following pieces of context to answer the user's question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), 
use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{question}")
])

In [ ]:
chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "summaries",
}

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # GPT-4를 사용하려면 model="gpt-4"

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    input_key="question"
)

In [ ]:
query = "소비자 부문 AI 활용사례 무엇인가요?"
query_result = chain.invoke({"question": query})
print(type(query_result))
print(query_result)

In [ ]:
print(query_result['result'])

In [ ]:
query_result['source_documents']

In [ ]:
for doc in query_result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

In [ ]:
bot_message = query_result['result']
for i, doc in enumerate(query_result['source_documents']):
    bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '
    print(bot_message)

In [ ]:
#%pip install -q gradio

In [34]:
from langchain.document_loaders import PyPDFLoader
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
import gradio as gr

# 1. PDF를 한 번만 로드하여 벡터 저장소 생성
def initialize_retriever():
    pdf_path = "../data/tutorial-korean.pdf"
    # PDF 파일을 로드하여 문서 객체로 변환
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # 문서를 일정한 크기로 분할 (chunk_size=1000, 중첩 없음)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, 
        chunk_overlap=0,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    texts = loader.load_and_split(text_splitter=text_splitter)

    # 문서의 텍스트를 벡터 임베딩으로 변환
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

    # 변환된 벡터를 FAISS 벡터 저장소에 저장
    vector_store = FAISS.from_documents(texts, embeddings)
    vector_store.save_local("../db/faiss_db")
    # 저장된 벡터를 검색할 수 있는 retriever 생성 (유사한 문서 2개 검색)
    retriever = vector_store.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": 6}
    )
    return retriever

# 2. 전역 retriever 생성 (앱 시작 시 한 번만 실행)
retriever = initialize_retriever()

retrieved_docs = retriever.invoke("코드패드는 무엇이고 어떻게 사용하나요?")
for doc in retrieved_docs:
    print(doc.page_content)

6. 코드패드 사용하기 ··················································· ······························································· ········· 21
216. 코드패드의 사용
BlueJ 코드패드는 자바 코드(표현식과 명령문)의 일부분을 쉽고 빠르게 평가할 수 있는 기능을 
제공합니다. 따라서, 코드패드는 자바언어로 작성된 프로그램 코드의 의미를 상세히 조사하거나 구문을 예증하고 시험하는데 사용할 수 있 습니다.
6.1. 코드패드 나타내기
요 약 코드패드에 명령문들을 입력하여 실행시킬 수 있습니다.
코드패드에서는 사용자가 이전에 입력한 내용들에 대한 이력을 저장하고 있습니다. 상하 화살표를 
이용하여 이전에 입력한 내용을 손쉽게 다시 입력시킬 수 있을 뿐만 아니라, 이전에 입력했던 내용을 수정하여 재입력시킬 수도 있습니다.   요 약 입력이력을 사용하기 위해서는 상하 화살표를 사용하십시오.
6.1. 코드패드 보기 ················································ ······························································· ··········· 21
코드패드 영역에는 표현식 또는 문장들을 입력할 수 있습니다. 키보드의 Enter를 누르면 
입력된 표현식 또는 문장이 라인단위로 평가되어 그 결과가 화면에 나타납니다.
그림 13 : 코드패드를 나타낸 메인화면  요 약  코드패드를 사용하기 위해 보기메뉴에서 코드패드보기(Show Code Pad)를 
 선택하십시오


In [ ]:

# 3. 채팅 응답 함수 (retriever를 재사용)
def chat_respond(message, chat_history):
    #  시스템 메시지 템플릿: 문서 요약을 기반으로 질문에 답변하도록 설정
    system_template = """당신은 BlueJ 프로그래밍 환경 전문가입니다. 
        아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

        문서 내용:
        {context}

        질문: {question}

        답변 규칙:
        1. 문서 내용만을 근거로 답변하세요
        2. 단계별 설명이 필요하면 순서대로 작성하세요  
        3. 구체적인 메뉴명, 버튼명을 포함하세요
        4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

        답변:
    """
    # 사용자 질문을 받아 최종 Prompt 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}")
    ])

    chain_type_kwargs = {
        "prompt": prompt, #  LLM이 사용할 프롬프트 지정
        "document_variable_name": "context", #  문서 요약 데이터를 LLM에 전달할 변수 이름
    }
    
    llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
    
    # RetrievalQA 체인 생성
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  #  문서를 하나의 큰 텍스트로 처리하는 방식
        retriever=retriever, #  유사 문서를 검색하는 retriever 연결
        return_source_documents=True, #  답변에 참조한 문서 정보 포함
        chain_type_kwargs=chain_type_kwargs,
        input_key="question" #  사용자 질문을 "question" 키로 전달
    )
    
    # LLM을 호출하여 질문에 대한 응답 생성
    query_result = chain.invoke({"question": message})

    # 모델의 답변을 가져오기
    bot_message = query_result['result']

    # 참조한 문서 정보를 응답에 추가
    for i, doc in enumerate(query_result['source_documents']):
        bot_message += f' [{i+1}] {doc.metadata.get("source", "Unknown")} (Page {doc.metadata.get("page", "N/A")})'

    # Gradio 채팅 기록 형식에 맞춰 응답을 저장
    chat_history.append({"role": "user", "content": message})  # 사용자 메시지 추가
    chat_history.append({"role": "assistant", "content": bot_message})  # 봇 응답 추가
    
    # 입력 창 초기화 및 갱신된 채팅 기록 반환
    return "", chat_history

# 4. # Gradio UI 생성 및 실행
with gr.Blocks() as demo:  
    # 채팅 창 (채팅 메시지를 표시하는 Gradio 컴포넌트)
    chatbot = gr.Chatbot(label="채팅창", type="messages")
    # 사용자 입력 텍스트 박스 (메시지를 입력하는 필드)
    msg = gr.Textbox(label="입력")
    # 초기화 버튼 (채팅 기록을 초기화하는 버튼)
    clear = gr.Button("초기화")

    # 사용자가 입력 필드에 메시지를 입력하면 chat_respond()가 실행됨
    #  - 입력한 메시지는 msg에서 가져옴
    #  - 기존 채팅 기록(chatbot)과 함께 전달됨
    #  - 응답을 받은 후, msg를 초기화하고 chatbot에 대화 내용을 추가
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # 초기화 버튼 클릭 시, 채팅 기록을 비우도록 설정
    clear.click(lambda: [], None, chatbot, queue=False)

# Gradio 앱 실행 (debug 모드 활성화)
demo.launch(debug=True)